In [1]:
from auto_fp8 import AutoFP8ForCausalLM, BaseQuantizeConfig

In [2]:
pretrained_model_dir = "meta-llama/Meta-Llama-3-8B-Instruct"
quantized_model_dir = "Meta-Llama-3-8B-Instruct-FP8-Dynamic"

In [3]:
# Define quantization config with static activation scales
quantize_config = BaseQuantizeConfig(quant_method="fp8", activation_scheme="dynamic")
# For dynamic activation scales, there is no need for calbration examples
examples = []

# Load the model, quantize, and save checkpoint
model = AutoFP8ForCausalLM.from_pretrained(pretrained_model_dir, quantize_config)


Loading model with the following kwargs: {'torch_dtype': 'auto', 'device_map': 'auto', 'cache_dir': None, 'force_download': False, 'proxies': None, 'resume_download': False, 'local_files_only': False, 'use_auth_token': None, 'revision': None, 'subfolder': '', '_commit_hash': None}


/opt/conda/envs/pytorch/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
model.quantize(examples)
model.save_quantized(quantized_model_dir)

Quantizing weights: 100%|██████████| 454/454 [00:00<00:00, 2878.16it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): FP8DynamicLinear()
          (k_proj): FP8DynamicLinear()
          (v_proj): FP8DynamicLinear()
          (o_proj): FP8DynamicLinear()
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): FP8DynamicLinear()
          (up_proj): FP8DynamicLinear()
          (down_proj): FP8DynamicLinear()
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=128256, bias=False)
)
Saving the model to Meta-Llama-3-8B-Instruct-FP8-Dynamic


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [1]:
from vllm import LLM
model = LLM(model="Meta-Llama-3-8B-Instruct-FP8-Dynamic/", tensor_parallel_size=4, max_model_len=1024)

INFO 07-08 15:06:24 config.py:698] Defaulting to use mp for distributed inference
INFO 07-08 15:06:24 llm_engine.py:169] Initializing an LLM engine (v0.5.1) with config: model='Meta-Llama-3-8B-Instruct-FP8-Dynamic/', speculative_config=None, tokenizer='Meta-Llama-3-8B-Instruct-FP8-Dynamic/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=fp8, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=Meta-Llama-3-8B-Instruct-FP8-Dynamic/, use_v2_block_manager=False, enable_prefix_caching=False)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(VllmWorkerProcess pid=54138) INFO 07-08 15:06:24 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=54139) INFO 07-08 15:06:24 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=54140) INFO 07-08 15:06:25 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
INFO 07-08 15:06:25 utils.py:741] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=54138) (VllmWorkerProcess pid=54139) INFO 07-08 15:06:25 pynccl.py:63] vLLM is using nccl==2.21.5
(VllmWorkerProcess pid=54140) INFO 07-08 15:06:25 utils.py:741] Found nccl from library libnccl.so.2
INFO 07-08 15:06:25 utils.py:741] Found nccl from library libnccl.so.2
INFO 07-08 15:06:25 utils.py:741] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=54138) (VllmWorkerProcess pid=54140) (VllmWorkerProcess pid=54139) INFO 07-08 15:06:25 pynccl.py:63] vLLM is using nccl==2.21.5
INFO 07-08 15:06:25 pynccl.py:63] vLLM is using nccl==2.21.5
INFO 07-08 15:06:25 pyn

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_SUPPORTED when calling `cublasLtMatmulAlgoGetHeuristic( ltHandle, computeDesc.descriptor(), Adesc.descriptor(), Bdesc.descriptor(), Cdesc.descriptor(), Ddesc.descriptor(), preference.descriptor(), 1, &heuristicResult, &returnedResult)`

In [ ]:
result = model.generate("Hello, my name is")

In [ ]:
# Execute in Shell
#%%sh
#python -m vllm.entrypoints.openai.api_server --model dwetzel/Llama-3-8B-Instruct_Orce_plus --tensor-parallel-size 4 --enable-chunked-prefill --dtype auto --quantization fp8 